In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset as HfDataset
import torch
from sklearn.metrics import roc_auc_score
from transformers import (
    AutoTokenizer,TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer)
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

import os

In [2]:
X_df = pd.read_csv("Data/jigsaw-toxic-comment-train.csv")
train_df, val_df = train_test_split(X_df,test_size=0.1,random_state=42)

In [3]:
#GPU 디바이스 세팅
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# TRANSFORMER 모델 
MODEL_NAME = "xlm-roberta-base"

# 타겟 레이블 컬럼들
TARGET_COLUMN = 'toxic'
TEXT_COLUMN = "comment_text"

#토큰나이저
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

#분류 모델
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,  
    problem_type="single_label_classification" 
)
model.to(device) # 모델을 GPU로 이동

class ToxicCommentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx] # 이제 2D 배열이 아닌 1D 배열에서 값을 가져옴

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt', 
        )
    
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long) 
        }

# 데이터셋 생성
train_texts = train_df[TEXT_COLUMN].values

# 1D 배열로 레이블 추출
train_labels = train_df[TARGET_COLUMN].values 
val_texts = val_df[TEXT_COLUMN].values
val_labels = val_df[TARGET_COLUMN].values 

train_dataset = ToxicCommentDataset(train_texts, train_labels, tokenizer)
val_dataset = ToxicCommentDataset(val_texts, val_labels, tokenizer)

# --- 👇 [새 단계 추가] 클래스 가중치 계산 ---
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# 'balanced' 모드는 레이블 빈도의 역수에 비례하여 가중치를 자동으로 계산합니다.
# (소수 클래스 '1'이 더 높은 가중치를 받게 됩니다)
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_labels), # [0, 1]
    y=train_labels
)

# 텐서로 변환하여 GPU로 이동
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

print(f"--- 클래스 불균형 해소를 위한 가중치 적용 ---")
print(f"계산된 클래스 가중치 (Class 0, Class 1 순서): {class_weights_tensor}")
print("Class 1(toxic)에 더 높은 페널티가 부여됩니다.")
# --- [새 단계 종료] ---

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report

def compute_metrics(p):
    preds_logits = p.predictions # (batch_size, 2) 형태의 로짓
    labels = p.label_ids # (batch_size,) 형태의 0 또는 1

    # 1. 로짓 -> 0/1 예측 (F1, Accuracy용)
    # 로짓 중 가장 큰 값의 인덱스(0 또는 1)를 예측값으로 사용
    preds_binary = np.argmax(preds_logits, axis=1)

    # 2. 로짓 -> 확률 (AUC용)
    softmax = torch.nn.Softmax(dim=1)
    # 클래스 1('toxic')에 대한 확률
    probs_class_1 = softmax(torch.tensor(preds_logits)).numpy()[:, 1]

    # 3. 메트릭 계산
    try:
        auc = roc_auc_score(labels, probs_class_1)
    except ValueError:
        auc = 0.5
    
    # F1 스코어 계산 (zero_division=0은 경고 대신 0.0 반환)
    # 'macro' F1: 클래스별 F1을 계산 후 단순 평균 (불균형 데이터 평가에 중요)
    f1_macro = f1_score(labels, preds_binary, average='macro', zero_division=0)
    # 'weighted' F1: 클래스별 F1을 계산 후 샘플 수로 가중 평균
    f1_weighted = f1_score(labels, preds_binary, average='weighted', zero_division=0)
    # F1 (class 1, toxic): 소수 클래스인 'toxic'만의 F1 (가장 중요할 수 있음)
    f1_toxic_class = f1_score(labels, preds_binary, pos_label=1, average='binary', zero_division=0)
    
    accuracy = accuracy_score(labels, preds_binary)

    # 로그에 남길 딕셔너리 반환
    return {
        "roc_auc": auc,
        "accuracy": accuracy,
        "f1_macro": f1_macro,           # <--- 이 지표를 모니터링할 예정
        "f1_weighted": f1_weighted,
        "f1_toxic_class": f1_toxic_class # <--- 소수 클래스 F1
    }

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- 클래스 불균형 해소를 위한 가중치 적용 ---
계산된 클래스 가중치 (Class 0, Class 1 순서): tensor([0.5529, 5.2258], device='cuda:0')
Class 1(toxic)에 더 높은 페널티가 부여됩니다.


In [6]:
from torch.nn import CrossEntropyLoss


# Trainer를 상속받아 compute_loss 메서드만 오버라이드합니다.
class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    # **kwargs를 추가하여 예상치 못한 추가 인자들을 모두 받습니다.
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        loss_fct = CrossEntropyLoss(weight=self.class_weights)
        
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir='./results',             
    num_train_epochs=1,                 
    per_device_train_batch_size=64,     
    per_device_eval_batch_size=256,   
    warmup_steps=100,                   
    weight_decay=0.01,                  
    logging_dir='./logs',               
    logging_steps=100,
    eval_steps=100,                  
    eval_strategy="steps",              
    save_strategy="steps",
    load_best_model_at_end=True,        
    metric_for_best_model="f1_macro",   
    greater_is_better=True,             
    fp16=True,                          
)

print("커스텀 WeightedTrainer를 사용하여 학습을 준비합니다.")
trainer = WeightedTrainer(  
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    class_weights=class_weights_tensor  
)

print("학습을 시작합니다...")
trainer.train()

print("학습 완료. 최종 평가를 수행합니다.")
eval_results = trainer.evaluate()
print(f"최종 검증 결과 (Validation Set): {eval_results}")

# 학습된 모델 저장
trainer.save_model("./best_toxic_model")
tokenizer.save_pretrained("./best_toxic_model")
print("최고 성능의 모델이 './best_toxic_model'에 저장되었습니다.")

커스텀 WeightedTrainer를 사용하여 학습을 준비합니다.
학습을 시작합니다...


/tmp/ipykernel_269962/142729437.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Step,Training Loss,Validation Loss,Roc Auc,Accuracy,F1 Macro,F1 Weighted,F1 Toxic Class
100,0.541100,0.374421,0.952656,0.910937,0.800756,0.920634,0.652591
200,0.376400,0.269798,0.967311,0.884634,0.770295,0.901417,0.608233
300,0.351200,0.266201,0.964513,0.942653,0.847539,0.944969,0.727118
400,0.266400,0.265615,0.971167,0.920734,0.819021,0.928793,0.683345
500,0.277900,0.215029,0.972543,0.919705,0.818256,0.928117,0.682469
600,0.262300,0.321059,0.956380,0.885753,0.773743,0.902544,0.614549
700,0.256800,0.268867,0.972827,0.929814,0.833137,0.935899,0.706125
800,0.274400,0.210802,0.976593,0.911832,0.808862,0.922368,0.668572
900,0.222800,0.227803,0.975357,0.939879,0.849466,0.943856,0.732803
1000,0.257900,0.205296,0.977611,0.892865,0.784126,0.908087,0.630914


학습 완료. 최종 평가를 수행합니다.


최종 검증 결과 (Validation Set): {'eval_loss': 0.42328763008117676, 'eval_roc_auc': 0.9755133376934639, 'eval_accuracy': 0.9556251397897562, 'eval_f1_macro': 0.8700489816170328, 'eval_f1_weighted': 0.9553704067242513, 'eval_f1_toxic_class': 0.764594209776934, 'eval_runtime': 45.6361, 'eval_samples_per_second': 489.853, 'eval_steps_per_second': 1.928, 'epoch': 1.0}
최고 성능의 모델이 './best_toxic_model'에 저장되었습니다.


In [7]:

# 이전에 학습하고 저장한 모델의 경로
MODEL_PATH = "./best_toxic_model" 

# 테스트 파일
TEST_FILE = "Data/test.csv" 

# 제출할 파일 이름
SUBMISSION_FILE = "submission22.csv"

TEXT_COLUMN = "content"

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 튜닝된 토큰나이저 로드
print(f"Loading model from {MODEL_PATH}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=2, 
    problem_type="single_label_classification"
)
model.to(device) 
model.eval() 

test_df = pd.read_csv(TEST_FILE)


# Pandas DataFrame을 Hugging Face Dataset 객체로 변환
test_dataset = HfDataset.from_pandas(test_df[[TEXT_COLUMN]])

# 테스트 데이터 토큰화
def tokenize_function(examples):
    return tokenizer(
        examples[TEXT_COLUMN],
        padding="max_length", # 모든 입력을 동일한 길이로 패딩
        truncation=True,      # 최대 길이를 초과하는 텍스트는 자름
        max_length=512        # 학습 시 사용했던 max_len과 동일하게 설정
    )

print("Tokenizing test data... (This may take a while)")

# .map()을 사용하여 모든 테스트 데이터에 토크나이징 함수를 일괄 적용
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# 모델이 입력으로 받지 않는 원본 텍스트 컬럼은 제거
tokenized_test_dataset = tokenized_test_dataset.remove_columns([TEXT_COLUMN])

# 추론(Inference)을 위한 최소한의 TrainingArguments 설정
inference_args = TrainingArguments(
    output_dir='./results_inference',       
    per_device_eval_batch_size=256,    # 배치 크기 (
    fp16=True if device == "cuda" else False, # GPU 사용 시 예측 속도 향상 (fp16)
    do_train=False,
    do_eval=False,
    do_predict=True,
)

# 모델과 추론 설정을 사용하여 Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=inference_args
)

# .predict() 메서드 호출하여 추론 실행
print("Running predictions...")
predictions = trainer.predict(tokenized_test_dataset)

# `predictions.predictions`에 모델의 원본 출력(logits)이 numpy 배열로 저장됨
raw_logits = predictions.predictions

Using device: cuda
Loading model from ./best_toxic_model...
Tokenizing test data... (This may take a while)


Map:   0%|          | 0/63812 [00:00<?, ? examples/s]

Running predictions...


In [8]:
print("Applying Softmax to get probabilities...")

softmax = torch.nn.Softmax(dim=1)
probs_tensor = softmax(torch.tensor(raw_logits))
probs_class_1 = probs_tensor.numpy()[:, 1] 

print("Converting probabilities to binary (0/1) predictions with threshold 0.5...")
binary_predictions = (probs_class_1 >= 0.5).astype(int) 

print(f"Creating submission file: {SUBMISSION_FILE}")

submission_df = pd.DataFrame(binary_predictions, columns=[TARGET_COLUMN])
submission_df['id'] = test_df['id']
submission_df = submission_df[['id'] + [TARGET_COLUMN]]
submission_df.to_csv(SUBMISSION_FILE, index=False)

print("Done. Submission file created successfully.")

Applying Softmax to get probabilities...
Converting probabilities to binary (0/1) predictions with threshold 0.5...
Creating submission file: submission22.csv
Done. Submission file created successfully.


In [9]:
from sklearn.metrics import classification_report, accuracy_score

ANSWER_FILE = "Data/test_labels.csv"


print(f"\nLoading answer key from {ANSWER_FILE} for evaluation...")
answer_df = pd.read_csv(ANSWER_FILE)

# 1. 예측(submission_df)과 정답(answer_df)을 'id' 기준으로 병합
#    submission_df의 'toxic' 열과 answer_df의 'toxic' 열을 비교합니다.
print("Merging predictions and answers...")
merged_df = pd.merge(
        submission_df, 
        answer_df, 
        on="id", 
        suffixes=('_pred', '_true') # 'toxic_pred'와 'toxic_true' 열이 생성됨
)

scored_df = merged_df[merged_df['toxic_true'] != -1].copy()

if scored_df.empty:
    print("오류: 정답지와 예측을 병합했으나 유효한 (non -1) 행이 없습니다.")
else:
    print(f"Total scored test samples: {len(scored_df)}")

        # 3. 평가를 위해 'toxic' 열의 예측(y_pred)과 정답(y_true) 배열 분리
    y_pred = scored_df['toxic_pred'].values
    y_true = scored_df['toxic_true'].values
        
        # 'toxic' 열에 대한 정확도 점수 계산
        
        # 'toxic' 열 정확도
    toxic_accuracy = accuracy_score(y_true, y_pred)
    print("\n--- 1. 'toxic' 열 정확도 (Accuracy) ---")
    print(f"('toxic' 레이블이 일치하는 샘플의 비율): {toxic_accuracy:.4f}")

        # 'toxic' 지표 (Precision, Recall, F1-Score)
    print("\n--- 2. 'toxic' 열 상세 리포트 (Precision, Recall, F1-Score) ---")
        
    report = classification_report(
            y_true, 
            y_pred,
            target_names=['toxic=0 (non-toxic)', 'toxic=1 (toxic)'], 
            zero_division=0
    )
    print(report)
print("Done. Submission file created successfully.")


Loading answer key from Data/test_labels.csv for evaluation...
Merging predictions and answers...
Total scored test samples: 63812

--- 1. 'toxic' 열 정확도 (Accuracy) ---
('toxic' 레이블이 일치하는 샘플의 비율): 0.7982

--- 2. 'toxic' 열 상세 리포트 (Precision, Recall, F1-Score) ---
                     precision    recall  f1-score   support

toxic=0 (non-toxic)       0.80      0.99      0.88     49402
    toxic=1 (toxic)       0.80      0.14      0.24     14410

           accuracy                           0.80     63812
          macro avg       0.80      0.57      0.56     63812
       weighted avg       0.80      0.80      0.74     63812

Done. Submission file created successfully.


In [10]:
train_df['comment_text'].apply(len).describe()

count    201194.000000
mean        391.209315
std         592.841466
min           1.000000
25%          93.000000
50%         203.000000
75%         431.000000
max        5000.000000
Name: comment_text, dtype: float64